#### Create data set: Hungarian and American names

In [2]:
import pandas as pd
import numpy as np
import unidecode

In [23]:
us_given = pd.read_csv('us_given.txt', header=None)
us_given.columns = ['g_name','gender', 'cnt']
us_given.g_name = us_given.g_name.str.lower()
us_given = us_given.drop(['gender', 'cnt'], axis=1)
us_given.head()

,g_name
0,﻿jennifer
1,amanda
2,jessica
3,melissa
4,sarah


In [42]:
us_given.shape

(19445, 1)

In [44]:
us_family = pd.read_csv('us_family.txt', header=None)
us_family.columns = ['f_name']
us_family.f_name = us_family.f_name.str.lower()
us_family.head()

,f_name
0,﻿smith
1,johnson
2,williams
3,brown
4,jones


In [45]:
us_family.shape

(162254, 1)

In [93]:
hun_given = pd.read_csv('hun_given.txt', header=None)
nms = [i.split(' ')[1].lower() for i in hun_given[0].values]
nms = list(set(nms))
nms = [ unidecode.unidecode(i) for i in nms]
hun_given = pd.DataFrame({'g_name':nms})
hun_given.head()

,g_name
0,adrienn
1,cintia
2,zsolt
3,roza
4,ervin


In [94]:
hun_family = pd.read_csv('hun_family.txt', header=None)
nms = [i.split(' ')[0].lower() for i in hun_family[0].values]
nms = list(set(nms))
nms = [ unidecode.unidecode(i) for i in nms]
hun_family = pd.DataFrame({'f_name':nms})
hun_family.head()

,f_name
0,dioszegi
1,janko
2,kelemen
3,borsodi
4,torma


In [46]:
#remove Hungarian names from us names

In [53]:
us_family = pd.merge(us_family,hun_family, indicator=True, how='outer')\
    .query('_merge=="left_only"')\
    .drop('_merge', axis=1)

us_given = pd.merge(us_given,hun_given, indicator=True, how='outer')\
    .query('_merge=="left_only"')\
    .drop('_merge', axis=1)

In [ ]:
#create american names

In [55]:
#shuffle rows
us_family = us_family.sample(frac=1).reset_index(drop=True)
us_given = us_given.sample(frac=1).reset_index(drop=True)

In [67]:
us_given = pd.concat([us_given]*10, ignore_index=True)
us_given = us_given[:len(us_family)]

In [79]:
us = pd.concat([us_family, us_given], axis=1, ignore_index=True)
us.columns = ['family', 'given']
us['hun'] = 0

us.head()

,family,given,hun
0,kanevsky,anand,0
1,heifort,taylor,0
2,masey,josha,0
3,cruzbautista,jordache,0
4,bystry,giovanna,0


In [80]:
us.shape

(161630, 3)

In [ ]:
#create hungarian names

In [95]:
#shuffle rows
hun_family = hun_family.sample(frac=1).reset_index(drop=True)
hun_given = hun_given.sample(frac=1).reset_index(drop=True)

In [96]:
print('family names: {}, given names: {}'.format(len(hun_family), len(hun_given)))

family names: 1227, given names: 263


In [97]:
hun_given = pd.concat([hun_given]*800, ignore_index=True)
hun_given = hun_given[:160000]

In [98]:
hun_family = pd.concat([hun_family]*170, ignore_index=True)
hun_family = hun_family[:160000]

In [99]:
hun = pd.concat([hun_family, hun_given], axis=1, ignore_index=True)
hun.columns = ['family', 'given']
hun['hun'] = 1
hun = hun.drop_duplicates()
hun.head()

,family,given,hun
0,bodnar,dorka,1
1,kiss,livia,1
2,weber,zsombor,1
3,csatlos,luca,1
4,benko,barnabas,1


In [103]:
names = pd.concat([hun, us], axis=0, ignore_index=True)
names.head(3)

,family,given,hun
0,bodnar,dorka,1
1,kiss,livia,1
2,weber,zsombor,1


In [104]:
names.hun.value_counts()

0    161630
1    159775
Name: hun, dtype: int64

In [105]:
names.shape

(321405, 3)

In [115]:
names = names.sample(frac=1).reset_index(drop=True)
names['full_name'] = names.family + ', ' + names.given

names.to_csv('names_dataset.csv', index=False)

In [4]:
names = pd.read_csv('/home/janos/Documents/hungarian_detector/names_dataset.csv')
names.head(3)

,family,given,hun,full_name
0,petrovics,bernadett,1,"petrovics, bernadett"
1,balatoni,irma,1,"balatoni, irma"
2,rusznyak,valeria,1,"rusznyak, valeria"


In [6]:
#remove NaNs and '?'s
names = names.dropna()
names.shape

(321395, 4)

In [9]:
ss = [i for i, nm in enumerate(names.full_name) if '?' in nm]
names = names.drop(names.index[ss])
names = names.reset_index(drop=True)
names.shape

In [11]:
names.to_csv('/home/janos/Documents/hungarian_detector/names_dataset.csv', index=False)